# <center>Título da Proposta</center>
---
## <center>Objeto da Proposta<center>

Sugestão para Título: 
Sugestão para Objeto:

Desenvolvido por:
* <b>George Ulguim Pedra</b> - gupbadabum@gmail.com
* <b>Cassia Lemos </b> - cassia.mg.lemos@gmail.com
* <b>Tatiane Moraes</b> - taticmsousa@gmail.com

Criado:     21-07-2022

Atualizado: 22-02-2023


---
## 0. BIBLIOTECAS E FUNÇÕES UTILIZADAS
--- 

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import csv
import shapefile as shp
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
from scipy.stats import norm

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
rcParams['text.usetex'] = True
rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'

np.random.seed(12345)

---
### 2. CONJUNTO DE DADOS
---

#### 2.1 PERFIL EPIDEMIOLÓGICO DA MALÁRIA

#### 2.2 MÉDIA ANUAL DA TEMPERATURA MÁXIMA

#### 2.3 MÉDIA ANUAL DO ÍNDICE SIMPLES DE INTENSIDADE DE PRECIPITAÇÃO

#### 2.4 MÉDIA ANUAL DA UMIDADE RELATIVA  

---
### 3. LEITURA DE DADOS
---

In [2]:
# Nome do Arquivo
path = '../2-Input/1-Proc/Dados_Malaria_Climaticos_20220621.csv'

data0 = pd.read_csv(path,delimiter=';',decimal='.',encoding='utf-8')

data1 = data0[['GEOCOD','FAT','50008-2020','50008-2020','50026-2020','50026-2030-O','50026-2030-P','50026-2050-O','50026-2050-P','50126-2020','50126-2030-O','50126-2030-P','50126-2050-O','50126-2050-P','50226-2020','50226-2030-O','50226-2030-P','50226-2050-O','50226-2050-P']]

data1.shape

(5570, 19)

##### 3.1 IDENTIFICAÇÃO DOS ATRIBUTOS DISPOSTOS NO CONJUNTO
* Informações:
    * GEOCOD (GeoCodigo) (Coluna 0)

* Fator Malária:
    * FAT (Coluna 1)

* Perfil Epidemiológico da Malária:
    * 50008-2020 (Coluna 2)

* Perfil Epidemiológico da Malária Categorizado - Binário:
    * 50008-2020 (Coluna 3)

* Dados de Temperatura:
    * 50026 - (2020, 2030-O,2030-P,2050-O,2050-P) (Colunas 4-9)


* Dados de Precipitação:
    * 50126 - (2020, 2030-O,2030-P,2050-O,2050-P) (Colunas 10-14)
    

* Dados de Umidade:    
    * 50226 - (2020, 2030-O,2030-P,2050-O,2050-P) (Colunas 15-19)


In [3]:
data1.head()

,GEOCOD,FAT,50008-2020,50008-2020,50026-2020,50026-2030-O,50026-2030-P,50026-2050-O,50026-2050-P,50126-2020,50126-2030-O,50126-2030-P,50126-2050-O,50126-2050-P,50226-2020,50226-2030-O,50226-2030-P,50226-2050-O,50226-2050-P
0,1100015,5,0.002054,0.002054,0.87,0.95,0.97,1.00,1.0,0.39,0.43,0.40,0.44,0.43,0.50,0.46,0.44,0.42,0.40
1,1100023,5,0.478520,0.478520,0.87,0.96,0.98,1.00,1.0,0.43,0.44,0.41,0.44,0.42,0.50,0.45,0.43,0.41,0.39
2,1100031,5,0.003146,0.003146,0.84,0.92,0.95,0.97,1.0,0.35,0.39,0.35,0.40,0.38,0.42,0.38,0.36,0.34,0.33
3,1100049,5,0.010898,0.010898,0.86,0.95,0.97,1.00,1.0,0.40,0.42,0.39,0.43,0.40,0.41,0.38,0.35,0.33,0.31
4,1100056,5,0.000000,0.000000,0.86,0.94,0.97,0.99,1.0,0.36,0.40,0.35,0.40,0.38,0.44,0.41,0.38,0.37,0.35


In [4]:
# Convertendo dados para numpy array
data = data1.to_numpy()
data.shape

(5570, 19)

##### 3.2 CATEGORIZANDO O DADO DO FAT

In [5]:
conditions1  = [ (data[:,1] >= 4),(data[:,1]>0) & (data[:,1]<4),(data[:,1]==0)]
choices1     = [2,1,0]
data[:,3] = np.select(conditions1, choices1, default=0)
print(data[:,3])

[2. 2. 2. ... 0. 0. 2.]


In [6]:
np.unique(data[:,3])

array([0., 1., 2.])

##### 3.3 SEPARAÇÃO DOS DADOS - TREINAMENTO - AVALIAÇÃO

In [7]:
# Variáveis explicativas
# Coluna 4  - Temperatura Máxima    - presente
# Coluna 9  - SDII                  - presente
# Coluna 14 - Umidade Relativa      - presente
atributos1 = [4,9,14] 		# 2020
atributos2 = [5,10,15]		# 2030-O
atributos3 = [6,11,16]		# 2030-P	
atributos4 = [7,12,17]		# 2050-O
atributos5 = [8,13,18]		# 2050-P

atriCodGeo = [0]
x = data[:,atributos1]

# Variáveis resposta
# Perfil Epidemiológico categorizado - multiclasse
y = data[:,1]

# Georeferência 
CodGeo=data[:,atriCodGeo]

X = preprocessing.normalize(x, axis=0, norm='max')

clas0Pos = np.where(y==0)[0]
x0 = X[clas0Pos,:]
y0 = y[clas0Pos]
geo0 = CodGeo[clas0Pos]

clas1Pos = np.where(y==1)[0]
x1 = X[clas1Pos,:]
y1 = y[clas1Pos]
geo1 = CodGeo[clas1Pos]

clas2Pos = np.where(y==2)[0]
x2 = X[clas2Pos,:]
y2 = y[clas2Pos]
geo2 = CodGeo[clas2Pos]

clas3Pos = np.where(y==3)[0]
x3 = X[clas3Pos,:]
y3 = y[clas3Pos]
geo3 = CodGeo[clas3Pos]

clas4Pos = np.where(y==4)[0]
x4 = X[clas4Pos,:]
y4 = y[clas4Pos]
geo4 = CodGeo[clas4Pos]


clas5Pos = np.where(y==5)[0]
x5 = X[clas5Pos,:]
y5 = y[clas5Pos]
geo5 = CodGeo[clas5Pos]

In [8]:
classes = [0,1,2,3,4,5]
comprimento = [len(clas0Pos),len(clas1Pos),len(clas2Pos),len(clas3Pos),len(clas4Pos),len(clas5Pos)]
percentual = [len(clas0Pos)/len(y),len(clas1Pos)/len(y),len(clas2Pos)/len(y),len(clas3Pos)/len(y),len(clas4Pos)/len(y),len(clas5Pos)/len(y)]

Info = pd.DataFrame({'Classe':classes,'Comprimento':comprimento,'Percentual':percentual})

Info.head(6)

,Classe,Comprimento,Percentual
0,0,4101,0.736266
1,1,283,0.050808
2,2,288,0.051706
3,3,310,0.055655
4,4,294,0.052783
5,5,294,0.052783


Conjunto de Dados completos, isto é, para os 5 recortes. Presente, 2030-O, 2030-P, 2050-O, 2050-P

In [9]:
DATA = preprocessing.normalize(data, axis=0, norm='max')
xPresente = DATA[:,atributos1]
x2030O = DATA[:,atributos2]
x2030P = DATA[:,atributos3]
x2050O = DATA[:,atributos4]
x2050P = DATA[:,atributos5]

print(xPresente)

[[0.87 0.39 0.5 ]
 [0.87 0.43 0.5 ]
 [0.84 0.35 0.42]
 ...
 [0.61 0.23 0.16]
 [0.64 0.33 0.11]
 [0.51 0.29 0.21]]


Devido ao fato das classes não estarem igualmente distríbuidas é necessário fazer um balanço para que o treinamento e avaliação não sejam tendênciosos. Dito isso, serão selecionados 9 valores de cada classe para o processo de treinamento e 6 valores destas mesmas classes serão considerados para avaliação

In [10]:
N = 283 # Menor comprimento dentre as classes

PercTeste = 0.66 # Percentual de dados para teste

pos1Ale = np.argsort(np.random.uniform(0,1,283))
pos2Ale = np.argsort(np.random.uniform(0,1,288))
pos3Ale = np.argsort(np.random.uniform(0,1,310))
pos4Ale = np.argsort(np.random.uniform(0,1,294))
pos5Ale = np.argsort(np.random.uniform(0,1,294))

pos_teste1 = pos1Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino1  = pos1Ale[0:np.int64(np.ceil(N*PercTeste))]

pos_teste2 = pos2Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino2  = pos2Ale[0:np.int64(np.ceil(N*PercTeste))]

pos_teste3 = pos3Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino3  = pos3Ale[0:np.int64(np.ceil(N*PercTeste))]

pos_teste4 = pos4Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino4  = pos4Ale[0:np.int64(np.ceil(N*PercTeste))]

pos_teste5 = pos5Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino5  = pos5Ale[0:np.int64(np.ceil(N*PercTeste))]

In [11]:
print(len(pos_treino1))
print(len(pos_treino2))
print(len(pos_treino3))
print(len(pos_treino4))
print(len(pos_treino5))

print(len(pos_teste1))
print(len(pos_teste2))
print(len(pos_teste3))
print(len(pos_teste4))
print(len(pos_teste5))

187
187
187
187
187
96
101
123
107
107


In [12]:
#Subconjunto de treino
yD1 = y1[ pos_treino1 ]
xD1 = x1[ pos_treino1 , :]
geo1treino = geo1[ pos_treino1 ]

yD2 = y2[ pos_treino2 ]
xD2 = x2[ pos_treino2 , :]
geo2treino = geo2[ pos_treino2 ]

yD3 = y3[ pos_treino3 ]
xD3 = x3[ pos_treino3 , :]
geo3treino = geo3[ pos_treino3 ]

yD4 = y4[ pos_treino4 ]
xD4 = x4[ pos_treino4 , :]
geo4treino = geo4[ pos_treino4 ]

yD5 = y5[ pos_treino5 ]
xD5 = x5[ pos_treino5 , :]
geo5treino = geo5[ pos_treino5 ]


In [13]:
# Seção de Treinamento
XD1=pd.DataFrame(xD1)    ;   YD1=pd.DataFrame(yD1) ;  GD1=pd.DataFrame(geo1treino)
XD2=pd.DataFrame(xD2)    ;   YD2=pd.DataFrame(yD2) ;  GD2=pd.DataFrame(geo2treino)
XD3=pd.DataFrame(xD3)    ;   YD3=pd.DataFrame(yD3) ;  GD3=pd.DataFrame(geo3treino)
XD4=pd.DataFrame(xD4)    ;   YD4=pd.DataFrame(yD4) ;  GD4=pd.DataFrame(geo4treino)
XD5=pd.DataFrame(xD5)    ;   YD5=pd.DataFrame(yD5) ;  GD5=pd.DataFrame(geo5treino)

XD=XD1.append(XD2)       ;    YD=YD1.append(YD2)   ;  GD=GD1.append(GD2)
XD=XD.append(XD3)        ;    YD=YD.append(YD3)    ;  GD=GD.append(GD3)
XD=XD.append(XD4)        ;    YD=YD.append(YD4)    ;  GD=GD.append(GD4)
XD=XD.append(XD5)        ;    YD=YD.append(YD5)    ;  GD=GD.append(GD5)

xD = XD.to_numpy()       ;    yD = YD.to_numpy()   ;  
print(yD.shape)
print(xD.shape)
np.unique(yD)

(935, 1)
(935, 3)


array([1., 2., 3., 4., 5.])

In [14]:
#Subconjunto de avaliação
yI1 = y1[ pos_teste1 ]
xI1 = x1[ pos_teste1 , :]
geo1teste = geo1[ pos_teste1 ]

yI2 = y2[ pos_teste2 ]
xI2 = x2[ pos_teste2 , :]
geo2teste = geo2[ pos_teste2 ]

yI3 = y3[ pos_teste3 ]
xI3 = x3[ pos_teste3 , :]
geo3teste = geo3[ pos_teste3 ]

yI4 = y4[ pos_teste4 ]
xI4 = x4[ pos_teste4 , :]
geo4teste = geo4[ pos_teste4 ]

yI5 = y5[ pos_teste5 ]
xI5 = x5[ pos_teste5 , :]
geo5teste = geo5[ pos_teste5 ]

In [15]:
# Seção de Avaliação
XI1=pd.DataFrame(xI1)    ;   YI1=pd.DataFrame(yI1) ;  GI1=pd.DataFrame(geo1teste)
XI2=pd.DataFrame(xI2)    ;   YI2=pd.DataFrame(yI2) ;  GI2=pd.DataFrame(geo2teste)
XI3=pd.DataFrame(xI3)    ;   YI3=pd.DataFrame(yI3) ;  GI3=pd.DataFrame(geo3teste)
XI4=pd.DataFrame(xI4)    ;   YI4=pd.DataFrame(yI4) ;  GI4=pd.DataFrame(geo4teste)
XI5=pd.DataFrame(xI5)    ;   YI5=pd.DataFrame(yI5) ;  GI5=pd.DataFrame(geo5teste)

XI=XI1.append(XI2)       ;    YI=YI1.append(YI2)   ;  GI=GI1.append(GI2)
XI=XI.append(XI3)        ;    YI=YI.append(YI3)    ;  GI=GI.append(GI3)
XI=XI.append(XI4)        ;    YI=YI.append(YI4)    ;  GI=GI.append(GI4)
XI=XI.append(XI5)        ;    YI=YI.append(YI5)    ;  GI=GI.append(GI5)

xI = XI.to_numpy()       ;  yI = YI.to_numpy()
print(xI.shape)
print(yI.shape)

(534, 3)
(534, 1)


In [16]:
# Conjunto de Treinamento
GD.columns = ['GeoCod']
YD.columns = ['Classe']
XD.columns = ['Temp','Sdii','Urel']
Treino = pd.concat([GD,YD,XD],axis=1)
Treino.to_csv('../4-Output/Dados_Treino_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

In [17]:
# Conjunto de Teste
GI.columns = ['GeoCod']
YI.columns = ['Classe']
XI.columns = ['Temp','Sdii','Urel']
Teste = pd.concat([GI,YI,XI],axis=1)
Teste.to_csv('../4-Output/Dados_Teste_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

---
### 4. MÉTODOS DE CLASSIFICAÇÃO
---

#### 4.1 REGRESSÃO SOFTMAX (LOGISTÍCA) - Multinível

4.1.1 Instanciação dos Critérios para seleção dos Hyper Parâmetros

In [18]:
parameters = {
    'penalty' : ['l1','l2','none','elasticnet'],
    'C'          : np.logspace(-3,3,7),
    'solver'   : ['newton-cg','lbfgs','liblinear'],
}
logreg=LogisticRegression(multi_class = 'multinomial')
logreg_cv=GridSearchCV(logreg,     # model
                      param_grid = parameters,  #hyperparameters
                      scoring = 'accuracy',     # metric for scoring
                      cv=5)                     # number of folds

In [19]:
logreg_cv.fit(xD,yD)

/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

GridSearchCV(cv=5, estimator=LogisticRegression(multi_class='multinomial'),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2', 'none', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='accuracy')

In [20]:
print("Tuned Hyperparameters :", logreg_cv.best_params_)
print("Accuracy :",logreg_cv.best_score_)

Tuned Hyperparameters : {'C': 0.001, 'penalty': 'none', 'solver': 'newton-cg'}
Accuracy : 0.4


4.1.2 Instanciação do método de Regressão Softmax

In [21]:
g2 = LogisticRegression(multi_class='multinomial', C=0.001,penalty='none',solver = 'newton-cg')

g2.fit(xD,yD)

yEstg2 = g2.predict(xI)
yEstg2prob = g2.predict_proba(xI)
kappa_g2 = cohen_kappa_score(yI[:,0],yEstg2)   
TAg2 = np.count_nonzero(yI[0] == yEstg2[:])/yI.shape[0]
print('Taxa de Acerto:  %f '%(TAg2))
print(kappa_g2)

Taxa de Acerto:  0.382022 
0.2366840696934569


/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
print(g2.intercept_)

[  5.08906591   4.08368918   4.37189144   1.39928692 -14.94393346]


In [23]:
print(g2.coef_)

[[-4.96270780e+00 -1.49184217e+00 -2.38526197e+00]
 [-4.16923409e+00 -2.72119779e-01 -1.92518084e+00]
 [-4.49491785e+00 -1.98135263e-01 -2.23643904e+00]
 [-1.69537253e+00  1.96148680e+00 -8.38546868e-01]
 [ 1.53222323e+01  6.10414200e-04  7.38542872e+00]]


4.1.3 Salvando o resultado para avaliação

In [24]:
# Tempo Presente - 1986-2005
G2P = pd.DataFrame(yEstg2, columns=['RLClass'])
G2Pb = pd.DataFrame(yEstg2prob,columns=['P1','P2','P3','P4','P5'])
ResultRL = pd.concat([G2P,G2Pb],axis=1)
ResultRL.to_csv('../4-Output/Resultado_RegLog_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

4.1.4 Calculando os resultados para os períodos de interesse

In [25]:
g2presente = g2.predict(xPresente)
g2presenteProb = g2.predict_proba(xPresente)
g22030O = g2.predict(x2030O)
g22030OProb = g2.predict_proba(x2030O)
g22030P = g2.predict(x2030P)
g22030PProb = g2.predict_proba(x2030P)
g22050O = g2.predict(x2050O)
g22050OProb = g2.predict_proba(x2050O)
g22050P = g2.predict(x2050P)
g22050PProb = g2.predict_proba(x2050P)

In [26]:
geocod = data[:,0]
GEOCOD=pd.DataFrame(geocod,columns=['GEOCOD'])

G2P = pd.DataFrame(g2presente, columns=['RLClass'])
G2Pb = pd.DataFrame(g2presenteProb,columns=['P1','P2','P3','P4','P5'])
ResultRLP = pd.concat([GEOCOD,G2P,G2Pb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Presente_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G230O  = pd.DataFrame(g22030O, columns=['RLClass'])
G230OPb = pd.DataFrame(g22030OProb,columns=['P1','P2','P3','P4','P5'])
ResultRLP = pd.concat([GEOCOD,G230O,G230OPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_2030_O_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G230P  = pd.DataFrame(g22030P, columns=['RLClass'])
G230PPb = pd.DataFrame(g22030PProb,columns=['P1','P2','P3','P4','P5'])
ResultRLP = pd.concat([GEOCOD,G230P,G230PPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_2030_P_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G250O  = pd.DataFrame(g22050O, columns=['RLClass'])
G250OPb = pd.DataFrame(g22050OProb,columns=['P1','P2','P3','P4','P5'])
ResultRLP = pd.concat([GEOCOD,G250O,G250OPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_2050_O_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G250P  = pd.DataFrame(g22050P, columns=['RLClass'])
G250PPb = pd.DataFrame(g22050PProb,columns=['P1','P2','P3','P4','P5'])
ResultRLP = pd.concat([GEOCOD,G250P,G250PPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_2050_P_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)


#### 4.2 REGRESSÃO LOGISTÍCA - Binária

4.2.1 SEPARAÇÃO DOS DADOS - TREINAMENTO - AVALIAÇÃO

In [27]:
# Variáveis explicativas
# Coluna 4  - Temperatura Máxima    - presente
# Coluna 9  - SDII                  - presente
# Coluna 14 - Umidade Relativa      - presente
atributos1 = [4,9,14] 		# 2020
atributos2 = [5,10,15]		# 2030-O
atributos3 = [6,11,16]		# 2030-P	
atributos4 = [7,12,17]		# 2050-O
atributos5 = [8,13,18]		# 2050-P

atriCodGeo = [0]
x = data[:,atributos1]

# Variáveis resposta
# Perfil Epidemiológico categorizado - multiclasse
y = data[:,3]

# Georeferência 
CodGeo=data[:,atriCodGeo]

X = preprocessing.normalize(x, axis=0, norm='max')

clas0Pos = np.where(y==0)[0]
x0 = X[clas0Pos,:]
y0 = y[clas0Pos]
geo0 = CodGeo[clas0Pos]

clas1Pos = np.where(y==1)[0]
x1 = X[clas1Pos,:]
y1 = y[clas1Pos]
geo1 = CodGeo[clas1Pos]

clas2Pos = np.where(y==2)[0]
x2 = X[clas2Pos,:]
y2 = y[clas2Pos]
geo2 = CodGeo[clas2Pos]

In [28]:
classes = [0,1,2]
comprimento = [len(clas0Pos),len(clas1Pos),len(clas2Pos)]
percentual = [len(clas0Pos)/len(y),len(clas1Pos)/len(y),len(clas2Pos)/len(y)]

Info = pd.DataFrame({'Classe':classes,'Comprimento':comprimento,'Percentual':percentual})

Info.head(6)

,Classe,Comprimento,Percentual
0,0,4101,0.736266
1,1,881,0.158169
2,2,588,0.105566


In [29]:
N = 588 # Menor comprimento dentre as classes

PercTeste = 0.66 # Percentual de dados para teste

pos1Ale = np.argsort(np.random.uniform(0,1,881))
pos2Ale = np.argsort(np.random.uniform(0,1,588))

pos_teste1 = pos1Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino1  = pos1Ale[0:np.int64(np.ceil(N*PercTeste))]

pos_teste2 = pos2Ale[np.int64(np.floor(N*PercTeste)):-1]
pos_treino2  = pos2Ale[0:np.int64(np.ceil(N*PercTeste))]

Devido ao fato das classes não estarem igualmente distríbuidas é necessário fazer um balanço para que o treinamento e avaliação não sejam tendênciosos. Dito isso, serão selecionados 389 valores de cada classe para o processo de treinamento e os valores restastes destas mesmas classes serão considerados para avaliação

In [30]:
print(len(pos_treino1))
print(len(pos_treino2))

print(len(pos_teste1))
print(len(pos_teste2))


389
389
492
199


In [31]:
#Subconjunto de treino
yD1 = y1[ pos_treino1 ]
xD1 = x1[ pos_treino1 , :]
geo1treino = geo1[ pos_treino1 ]

yD2 = y2[ pos_treino2 ]
xD2 = x2[ pos_treino2 , :]
geo2treino = geo2[ pos_treino2 ]


In [32]:
# Seção de Treinamento
XD1=pd.DataFrame(xD1)    ;   YD1=pd.DataFrame(yD1) ;  GD1=pd.DataFrame(geo1treino)
XD2=pd.DataFrame(xD2)    ;   YD2=pd.DataFrame(yD2) ;  GD2=pd.DataFrame(geo2treino)

XD=XD1.append(XD2)       ;    YD=YD1.append(YD2)   ;  GD=GD1.append(GD2)

xD = XD.to_numpy()       ;    yD = YD.to_numpy()   ;  
print(yD.shape)
print(xD.shape)
np.unique(yD)

(778, 1)
(778, 3)


array([1., 2.])

In [33]:
#Subconjunto de avaliação
yI1 = y1[ pos_teste1 ]
xI1 = x1[ pos_teste1 , :]
geo1teste = geo1[ pos_teste1 ]

yI2 = y2[ pos_teste2 ]
xI2 = x2[ pos_teste2 , :]
geo2teste = geo2[ pos_teste2 ]

In [34]:
# Seção de Avaliação
XI1=pd.DataFrame(xI1)    ;   YI1=pd.DataFrame(yI1) ;  GI1=pd.DataFrame(geo1teste)
XI2=pd.DataFrame(xI2)    ;   YI2=pd.DataFrame(yI2) ;  GI2=pd.DataFrame(geo2teste)

XI=XI1.append(XI2)       ;    YI=YI1.append(YI2)   ;  GI=GI1.append(GI2)

xI = XI.to_numpy()       ;  yI = YI.to_numpy()
print(xI.shape)
print(yI.shape)

(691, 3)
(691, 1)


In [35]:
# Conjunto de Treinamento
GD.columns = ['GeoCod']
YD.columns = ['Classe']
XD.columns = ['Temp','Sdii','Urel']
Treino = pd.concat([GD,YD,XD],axis=1)
Treino.to_csv('../4-Output/Dados_Treino_Binario_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

In [36]:
# Conjunto de Teste
GI.columns = ['GeoCod']
YI.columns = ['Classe']
XI.columns = ['Temp','Sdii','Urel']
Teste = pd.concat([GI,YI,XI],axis=1)
Teste.to_csv('../4-Output/Dados_Teste_Binario_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

4.2.2 Instanciação do método de Regressão Logistíca - Binária

In [37]:
parameters = {
    'penalty' : ['l1','l2','none','elasticnet'],
    'C'          : np.logspace(-3,3,7),
    'tol'     : [0.000001,0.00001,0.0001],
    'solver'   : ['newton-cg','lbfgs','liblinear','newton-cholesky','sag','saga'],
}
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,     # model
                      param_grid = parameters,  #hyperparameters
                      scoring = 'accuracy',     # metric for scoring
                      cv=2,                     # number of folds
                      error_score=0)

In [38]:
logreg_cv.fit(xD,yD)

/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

GridSearchCV(cv=2, error_score=0, estimator=LogisticRegression(),
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2', 'none', 'elasticnet'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear',
                                    'newton-cholesky', 'sag', 'saga'],
                         'tol': [1e-06, 1e-05, 0.0001]},
             scoring='accuracy')

In [39]:
print("Tuned Hyperparameters :", logreg_cv.best_params_)
print("Accuracy :",logreg_cv.best_score_)

Tuned Hyperparameters : {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg', 'tol': 1e-06}
Accuracy : 0.7455012853470437


In [40]:
gb2 = LogisticRegression(C=0.001,penalty='none',solver = 'newton-cg',tol=1e-06)

gb2.fit(xD,yD)

ygb2 = gb2.predict(xI)
ygb2prob = gb2.predict_proba(xI)
kappa_gb2 = cohen_kappa_score(yI[:,0],ygb2)   
TAgb2 = np.count_nonzero(yI[0] == ygb2[:])/yI.shape[0]
print('Taxa de Acerto:  %f '%(TAgb2))
print('Kappa Cohen: %f  '%(kappa_gb2))

Taxa de Acerto:  0.643994 
Kappa Cohen: 0.502159  


/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/home/hellgate/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [41]:
print(gb2.intercept_)

[-6.36263286]


In [42]:
print(gb2.coef_)

[[6.32401723 2.17159752 3.5367569 ]]


In [43]:
def logit_pvalue(model, x):
    """ Calculate z-scores for scikit-learn LogisticRegression.
    parameters:
        model: fitted sklearn.linear_model.LogisticRegression with intercept and large C
        x:     matrix on which the model was fit
    This function uses asymtptics for maximum likelihood estimates.
    """
    p = model.predict_proba(x)
    n = len(p)
    m = len(model.coef_[0]) + 1
    coefs = np.concatenate([model.intercept_, model.coef_[0]])
    x_full = np.matrix(np.insert(np.array(x), 0, 1, axis = 1))
    ans = np.zeros((m, m))
    for i in range(n):
        ans = ans + np.dot(np.transpose(x_full[i, :]), x_full[i, :]) * p[i,1] * p[i, 0]
    vcov = np.linalg.inv(np.matrix(ans))
    se = np.sqrt(np.diag(vcov))
    t =  coefs/se  
    p = (1 - norm.cdf(abs(t))) * 2
    return p

In [44]:
print(logit_pvalue(gb2, xD))

[0.00000000e+00 0.00000000e+00 4.24631774e-03 8.88178420e-16]


In [45]:
G2P = pd.DataFrame(ygb2,columns=['RLClass'])
G2Pb = pd.DataFrame(ygb2prob,columns=['P1','P2'])
ResultRL = pd.concat([G2P,G2Pb],axis=1)
ResultRL.to_csv('../4-Output/Resultado_Teste_RegLog_Binaria_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

In [46]:
g2presente = gb2.predict(xPresente)
g2presenteProb = gb2.predict_proba(xPresente)
g22030O = gb2.predict(x2030O)
g22030OProb = gb2.predict_proba(x2030O)
g22030P = gb2.predict(x2030P)
g22030PProb = gb2.predict_proba(x2030P)
g22050O = gb2.predict(x2050O)
g22050OProb = gb2.predict_proba(x2050O)
g22050P = gb2.predict(x2050P)
g22050PProb = gb2.predict_proba(x2050P)

In [49]:
geocod = data[:,0]
GEOCOD=pd.DataFrame(geocod,columns=['GEOCOD'])

G2P = pd.DataFrame(g2presente, columns=['RLClass'])
G2Pb = pd.DataFrame(g2presenteProb,columns=['P1','P2'])
ResultRLP = pd.concat([GEOCOD,G2P,G2Pb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Binario_Presente_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G230O  = pd.DataFrame(g22030O, columns=['RLClass'])
G230OPb = pd.DataFrame(g22030OProb,columns=['P1','P2'])
ResultRLP = pd.concat([GEOCOD,G230O,G230OPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Binario_2030_O_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G230P  = pd.DataFrame(g22030P, columns=['RLClass'])
G230PPb = pd.DataFrame(g22030PProb,columns=['P1','P2'])
ResultRLP = pd.concat([GEOCOD,G230P,G230PPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Binario_2030_P_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G250O  = pd.DataFrame(g22050O, columns=['RLClass'])
G250OPb = pd.DataFrame(g22050OProb,columns=['P1','P2'])
ResultRLP = pd.concat([GEOCOD,G250O,G250OPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Binario_2050_O_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)

G250P  = pd.DataFrame(g22050P, columns=['RLClass'])
G250PPb = pd.DataFrame(g22050PProb,columns=['P1','P2'])
ResultRLP = pd.concat([GEOCOD,G250P,G250PPb],axis=1)
ResultRLP.to_csv('../4-Output/Resultado_RegLog_Binario_2050_P_20230301.csv',sep=';',encoding='utf-8',index=False,header=True)